# Implementation of a custom ABC

In [1]:
import time
import reprlib

class clock:
    DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'
    
    def __init__(self, fmt=None):
        if fmt:
            self.fmt = fmt
        else:
            self.fmt = clock.DEFAULT_FMT
            
    def __call__(self, func):
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = reprlib.repr(_result)
            print(self.fmt.format(**locals()))
            return _result
        return clocked

In [2]:
import abc
import bisect

class Tombola(abc.ABC):

    @abc.abstractmethod
    def load(self, iterable):
        """Add items from an iterable."""

    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it.

        This method should raise `LookupError` when the instance is empty.
        """

    def loaded(self):
        """Return `True` if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())
    
    @clock(fmt='[{elapsed:0.3f}s] {name}')
    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:
            try:
                bisect.insort(items, self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(items)

In [3]:
import random

class BingoCage(Tombola):

    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):
        return self.pick()

In [4]:
l1 = [random.randint(0,1000000) for i in range(0,300000)]
b1 = BingoCage(l1)

In [5]:
reprlib.repr(b1.inspect())

[10.981s] inspect


'(0, 4, 8, 12, 18, 31, ...)'

In [6]:
@clock(fmt='[{elapsed:0.3f}s] {name}')
def inspect_v2(self):
    """Return a sorted tuple with the items currently inside."""
    items = []
    while True:
        try:
            items.append(self.pick())
        except LookupError:
            break
    self.load(items)
    return tuple(sorted(items))

Tombola.inspect = inspect_v2

#neu erstellen da in b1 list sortiert gespeichert wurde
b2 = BingoCage(l1)
reprlib.repr(b2.inspect())

[0.356s] inspect_v2


'(0, 4, 8, 12, 18, 31, ...)'

Bisect insort is very slow (N for every item so complete list -> N\*N) if there is no need for a sorted list after every insert, inserting all the elements and then sortin with sorted(N\*log(n)) is alot faster.

In [7]:
class LotteryBlower(Tombola):

    def __init__(self, iterable):
        self._balls = list(iterable)

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty BingoCage')
        return self._balls.pop(position)

    def loaded(self):
        return bool(self._balls)

    @clock(fmt='[{elapsed:0.3f}s] {name}')
    def inspect(self):
        return tuple(sorted(self._balls))

In [9]:
lb1 = LotteryBlower(l1)
reprlib.repr(lb1.inspect())

[0.052s] inspect


'(0, 4, 8, 12, 18, 31, ...)'

significantly faster since no loading etc. is done.

In [15]:
from random import randrange

@Tombola.register
class TomboList(list):

    def pick(self):
        if self:
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')

    load = list.extend

    def loaded(self):
        return bool(self)

    def inspect(self):
        return tuple(sorted(self))

# alternative to the Decorator Tombola.register(TomboList)  
print('issubclass: ' + str(issubclass(TomboList, Tombola)))

t = TomboList(range(100))
print('isinstance: ' + str(isinstance(t, Tombola)))

issubclass: True
isinstance: True


In [16]:
TomboList.__mro__
# TomboList inheriits no methods of Tombola

(__main__.TomboList, list, object)

In [17]:
LotteryBlower.__mro__

(__main__.LotteryBlower, __main__.Tombola, abc.ABC, object)

# Protocols

In [28]:
from typing import TypeVar, Protocol

T = TypeVar('T')

class Repeatable(Protocol):
    def __mul__(self: T, repeat_count: int) -> T: ...

RT = TypeVar('RT', bound=Repeatable)

def double(x: RT) -> RT:
    return x * 2

double((2,))

(2, 2)

Fraction(4, 5)